In [36]:
import os
import sys
from pathlib import Path
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from shapely.geometry import Point

### change directory to import more scripts
sys.path.append(str(Path().resolve().parents[0]))

from utils import tools
import generate_wind

if os.getcwd().split(os.sep)[-1] == 'evaluation':
    os.chdir('..')

In [11]:
config = tools.load_config("config.yaml")

val_dir = 'data/Trianel_Benchmark'
synth_dir = os.path.join(config['data']['synth_dir'], 'ex_noise', 'wind_age')
turbine_dir = config['data']['turbine_dir']
turbine_power = config['data']['turbine_power']
turbine_path = os.path.join(turbine_dir, turbine_power)
specs_path = config['data']['turbine_specs']
specs_path = os.path.join(turbine_dir, specs_path)
cp_path = config["data"]["turbine_cp"]
cp_path = os.path.join(turbine_dir, cp_path)
results_dir = 'results'
geojsn_dir = 'misc/geoBoundaries-DE.geojson'
os.makedirs(results_dir, exist_ok=True)

features = config['features']

params = config['wind_params']

mapping = {'power_t1': 'Enercon E-70 E4 2.300',
           'power_t2': 'Enercon E-82 E2 2.000',
           'power_t3': 'Enercon E-115 2.500',
           'power_t4': 'Vestas V90',
           'power_t5': 'Vestas V112-3.45',
           'power_t6': 'Vestas V80-1.8'
           }

In [9]:
files = os.listdir(synth_dir)
station_files_synthetic = [f for f in files if 'parameter' not in f and f.endswith('.csv')]
station_files_synthetic.sort()
print(len(station_files_synthetic), 'station files in synthetic data found')
turbine_params = pd.read_csv(os.path.join(synth_dir, 'turbine_parameter.csv'), sep=';')
wind_params = pd.read_csv(os.path.join(synth_dir, 'wind_parameter.csv'), sep=';', dtype={"park_id": str})

raw_power_curves, cp_curves, specs = generate_wind.get_turbines(turbine_path=turbine_path,
                                                        cp_path=cp_path,
                                                        specs_path=specs_path,
                                                        params=params)
power_curves = pd.DataFrame()
for turbine in raw_power_curves.columns:
    power_curve = generate_wind.interpolate(power_curve=raw_power_curves[turbine],
                                            cut_out=specs[turbine]['cut_out'])
    power_curves[turbine] = power_curve
power_curves /= 1000 # convert to kW

110 station files in synthetic data found


In [51]:
masterdata = pd.read_csv(os.path.join(val_dir, 'masterdata_wind_20240601.csv'), sep=';')
meterdata = pd.read_csv(os.path.join(val_dir, 'meterdata_wind_20240601.csv'), sep=';')
meterdata['Zeit'] = pd.to_datetime(meterdata['Zeit'], utc=True)
meterdata.set_index('Zeit', inplace=True)

In [55]:
m = folium.Map(location=[51.0, 10.0], zoom_start=6)

# --- 4. Punkte aus df1: rote Marker mit Tooltip ---
for _, row in wind_params.iterrows():
    folium.CircleMarker(
        location=[row['longitude'], row['latitude']],
        radius=5,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.8,
        tooltip=f"Park ID: {row['park_id']}<br>Lat: {row['longitude']:.4f}, Lon: {row['latitude']:.4f}"
    ).add_to(m)

# --- 5. Punkte aus df2: blaue Marker mit Tooltip ---
for _, row in masterdata.iterrows():
    folium.CircleMarker(
        location=[row['Koordinaten Breite (WGS 84)'], row['Koordinaten Länge (WGS 84)']],
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.8,
        tooltip=f"Lat: {row['Koordinaten Breite (WGS 84)']:.4f}, Lon: {row['Koordinaten Länge (WGS 84)']:.4f}"
    ).add_to(m)

# --- 6. Karte anzeigen (in Jupyter) ---
m

In [53]:
lon1 = 9.793
lon2 = 9.778
lat1 = 52.9604
lat2 = 52.9383

dx = 71.5 * (lon1 - lon2)
dy = 111.3 * (lat1 - lat2)

distance = np.sqrt(dx * dx + dy * dy)

print(f'Distance is {distance} km.')

Distance is 2.6833799438209995 km.


In [47]:
masterdata[masterdata.Parkname == 'Windpark Jettebruch']

,Parkname,ParkID,EEG Anlagenschlüssel,Malo-ID,Installierte Leistung (kW),Hersteller,Typ,Nabenhöhe (m),Rotordurchmesser (m),Nachtabregelung,Nachtabregelung von (CET),Nachtabregelung bis (CET),Nachtabregelung max kW,Koordinaten Breite (WGS 84),Koordinaten Länge (WGS 84),Abregelungsmodus,Abregelung bis minimal (kW/%)
24,Windpark Jettebruch,20012091,E218780100000E0000000001753300010,50421572237,1800.0,ENERCON,E-66/18.70 1800,65.0,70.0,False,NaN,NaN,NaN,52.938280,9.778037,LIMITABLE,NaN
25,Windpark Jettebruch,20012091,E218780100000E0000000001753300020,50421572237,1800.0,ENERCON,E-66/18.70 1800,65.0,70.0,False,NaN,NaN,NaN,52.935700,9.777723,LIMITABLE,NaN
26,Windpark Jettebruch,20012091,E218780100000E0000000001753300030,50421572237,1800.0,ENERCON,E-66/18.70 1800,65.0,70.0,False,NaN,NaN,NaN,52.933674,9.777091,LIMITABLE,NaN
27,Windpark Jettebruch,20012091,E218780100000E0000000001753300040,50421572237,1800.0,ENERCON,E-66/18.70 1800,65.0,70.0,False,NaN,NaN,NaN,52.931110,9.774547,LIMITABLE,NaN
28,Windpark Jettebruch,20012091,E218780100000E0000000001753300050,50421572237,1800.0,ENERCON,E-66/18.70 1800,65.0,70.0,False,NaN,NaN,NaN,52.932053,9.769567,LIMITABLE,NaN


In [13]:
meterdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154849 entries, 0 to 154848
Data columns (total 16 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Zeit         154849 non-null  object 
 1   50082792290  154837 non-null  float64
 2   50361225962  0 non-null       float64
 3   50414611745  100321 non-null  float64
 4   50421485125  49633 non-null   float64
 5   50421572237  100297 non-null  float64
 6   50446515501  100321 non-null  float64
 7   50454932698  100321 non-null  float64
 8   50483800543  100321 non-null  float64
 9   50646350971  49633 non-null   float64
 10  50646350997  49633 non-null   float64
 11  50646396058  49633 non-null   float64
 12  50852909827  154837 non-null  float64
 13  51164756609  14597 non-null   float64
 14  51279106971  154837 non-null  float64
 15  51443998641  119709 non-null  float64
dtypes: float64(15), object(1)
memory usage: 18.9+ MB
